# LangChain Ensemble retriever 를 이용하여 OpenSearch Hybrid 검색을 통한 RAG
> 이 노트북은  SageMaker Studio* **`Data Science 3.0`** kernel 및 ml.t3.medium 인스턴스에서 테스트 되었습니다.
---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

### 선수조건
- 이 노트북은 이전 노트북인 "02_KR_RAG_OpenSearch_Claude.ipynb" 이 완료 되었다고 가정 합니다.
    - 오픈서치 인텍스 관련 정보를 참조 합니다.

### Methods and Resources for Hybrid search with Re-ranking 
- Score Normalization
    - [MinMax based](https://towardsdatascience.com/text-search-vs-vector-search-better-together-3bd48eb6132a)
    - [z-socre based](https://towardsdatascience.com/hybrid-search-2-0-the-pursuit-of-better-search-ce44d6f20c08)
- Reciprocal Rank Fusion (RRF)
    - [Paper](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf)
    - [Description](https://medium.com/@sowmiyajaganathan/hybrid-search-with-re-ranking-ff120c8a426d)
- [LangChain API for Ensemble Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble)

### 설정

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결해야 합니다.



In [2]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

# 1. Bedrock Client 생성

In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


# 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

## LLM 로딩 (Claude-v2)

In [4]:
from langchain.llms.bedrock import Bedrock

In [5]:
# - create the Anthropic Model
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={'max_tokens_to_sample': 512}
)
llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7fde44228550>, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 512})

## Embedding 모델 선택

In [6]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):
    if is_bedrock_embeddings:

        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings

        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )
        print("Bedrock Embeddings Model Loaded")

    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )        
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")
    
    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [7]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)    

Bedrock Embeddings Model Loaded


# 3. LangChain OpenSearch VectorStore 생성 
## 선수 조건
- 이전 노트북 02_1_KR_RAG_OpenSearch_Keyword.ipynb 또는 02_1_KR_RAG_OpenSearch_Semantic.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.

## 오픈 서치 도메인 및 인증 정보 세팅

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

#### [중요] 아래에 OpenSearch ID/PW , opensearch_domain_endpoint 를 입력을 해주세요.

In [8]:
os.environ["OpenSearch_UserName"] = "Type ID" 
os.environ["OpenSearch_UserPassword"] = "Type PW" 

rag_user_name = os.environ["OpenSearch_UserName"]
rag_user_password = os.environ["OpenSearch_UserPassword"]

In [9]:
opensearch_domain_endpoint = "<Tye OpenSearch Domain Endpoint>"


http_auth = (rag_user_name, rag_user_password) # Master username, Master password

index_name = "genai-demo-index-v1-with-tokenizer"

## LangChain OpenSearch VectorStore 생성

In [10]:
from langchain.vectorstores import OpenSearchVectorSearch

vectro_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2"
)

## OpenSearch Client 생성

In [11]:
from utils.rag import create_aws_opensearch_client, check_if_index_exists, delete_index

In [12]:
aws_client = create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 은전한잎 형태소 분석기 (seunjeon_tokenizer) 사용하기 in Opensearch
- "02_1_KR_RAG_OpenSearch_Keyword.ipynb"에서 만들어 두었던 tokenization index 활용
    - 형태소 분석기에 대한 자세한 사항은 02_1_KR_RAG_OpenSearch_Keyword.ipynb 참고

### 인덱스 확인 (tokenization 확인)

In [13]:
from pprint import pprint

In [14]:
index_info = aws_client.indices.get(index=index_name)
pprint (index_info)

{'genai-demo-index-v1-with-tokenizer': {'aliases': {},
                                        'mappings': {'properties': {'metadata': {'properties': {'row': {'type': 'long'},
                                                                                                'source': {'fields': {'keyword': {'ignore_above': 256,
                                                                                                                                  'type': 'keyword'}},
                                                                                                           'type': 'text'},
                                                                                                'timestamp': {'type': 'float'},
                                                                                                'type': {'fields': {'keyword': {'ignore_above': 256,
                                                                                                                         

# 4. 오픈 서치에 "유사 서치" 검색
- query 를 제공해서 실제로 유사한 내용이 검색이 되는지를 확인 합니다.



- similarity_search_with_score API 정보
    - [API: similarity_search_with_score](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html#langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.similarity_search)

In [15]:
import copy
from langchain.schema import Document
from langchain import PromptTemplate
from operator import itemgetter

## (1) OpenSearch Vector 검색 (Semantic search)

### 프롬프트 템플릿 생성

In [17]:
from utils.rag import create_bool_filter, run_RetrievalQA, show_context_used
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

### [TIP] Prompt의 instruction의 경우 한글보다 **영어**로 했을 때 더 좋은 결과를 얻을 수 있습니다.

In [34]:
# prompt_template = """
# \n\nHuman: 다음 문맥의 Information을 사용하여 고객 서비스 센터 직원처럼, 마지막 질문에 대한 목차 형식으로 답변을 제공하세요. 응답을 모르면 모른다고 말하고 응답을 만들려고 하지 마세요.

# {context}

# Question: {question}
# \n\nAssistant:"""

prompt_template = """


Human: Here is the context, inside <context></context> XML tags.    

<context>
{context}
</context>

Only using the contex as above, answer the following question with the rules as below:
    - Don't insert XML tag such as <context> and </context> when answering.
    - Write as much as you can
    - Be courteous and polite
    - Only answer the question if you can find the answer in the context with certainty.

Question:
{question}

If the answer is not in the context, just say "주어진 내용에서 관련 답변을 찾을 수 없습니다."


Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


### 필터 생성

In [35]:
filter01 = "인터넷뱅킹"
# filter01 = "인증서"
filter02 = "신한은행"
# filter02 = "아마존은행"

# query = "홈페이지 이용자아이디 여러 개 사용할 수 있나요?"
query = "타기관OTP 등록 방법 알려주세요"

boolean_filter = create_bool_filter(filter01, filter02)
pprint (boolean_filter)

{'bool': {'filter': {'term': {'metadata.source': '신한은행'}},
          'must': [{'match': {'metadata.type': '인터넷뱅킹'}}]}}


### Retriever for semantic search 정의

In [36]:
opensearch_semantic_retriever = vectro_db.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k":5,
        "boolean_filter":boolean_filter
    }
)

In [37]:
chain = load_qa_chain(
    llm=llm_text,
    chain_type="stuff",
    prompt=PROMPT,
    verbose=True
)

In [38]:
search_semantic_result = opensearch_semantic_retriever.get_relevant_documents(query)

answer = chain.run(
    input_documents=search_semantic_result,
    question=query
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Here is the context, inside <context></context> XML tags.    

<context>
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  
- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록
 
 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.

ask: 인터넷뱅킹에서 모바일OTP 인증요청 했는데 푸시메세지가 오지 않아요. 어떻게 해야하나요?
Information: 신한 쏠(SOL) 푸시알림 수신에 동의를 하신경우 푸시를 받을수 있습니다. 경로 : 신한 쏠(SOL) 로그인 > 전체메뉴 > 설정/인증  > 일반 > 알림 > 금융정보 알림 항목을 수신동의로 설정 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.

ask: 모바일 OTP 사용하고 있는데 인터넷뱅킹에서는 어떻게 이용하는건가요?
Information: 인터넷뱅킹 거래시 보안매체 입력단계에 [인증요청] 버튼 클릭하면 핸드폰으로 푸시메세지가 전달됩니다. 
푸시메세지 터치하여 모바일 OTP 비밀번호 6자리 입력, 인증처리 완료후 인터넷뱅킹에서 이후의 절차를 진행하시면 됩니다.
※ 스마트폰에 모바일 OTP 푸시(PUSH) 메세지가 오지 않는 경우 메뉴 직접 실행 
- 신한 쏠(SOL) [로그인 하지 않음]> 전체메뉴> 설정/인증> 보안매체 전체보기> 모바일OTP> 인증
기타 궁금하신 내용은 신한은행 고객

In [39]:
show_context_used(search_semantic_result)

-----------------------------------------------
1. Chunk: 279 Characters
-----------------------------------------------
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다.
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록
- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록

 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
metadata:
 {'source': '신한은행', 'row': 3, 'type': '인터넷뱅킹', 'timestamp': 1696832766.2917683}
-----------------------------------------------
2. Chunk: 225 Characters
-----------------------------------------------
ask: 인터넷뱅킹에서 모바일OTP 인증요청 했는데 푸시메세지가 오지 않아요. 어떻게 해야하나요?
Information: 신한 쏠(SOL) 푸시알림 수신에 동의를 하신경우 푸시를 받을수 있습니다. 경로 : 신한 쏠(SOL) 로그인 > 전체메뉴 > 설정/인증  > 일반 > 알림
> 금융정보 알림 항목을 수신동의로 설정 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
metadata:
 {'source': '신한은행', 'row': 11, 'type': '인터넷뱅킹', 'timestamp': 1696832766.2917902}
-----------------------------------------------
3. Chunk: 342 Characters
---------

## (2) OpenSearch Keyword 검색
- OpenSearch 에 아래와 같은 Query 를 실행하여 결과 받는 것을 구현합니다.

In [45]:
from utils.rag import BM25OpenSearchRetriever
from langchain.chains.question_answering import load_qa_chain

### Retriever for keyword search 정의

In [46]:
opensearch_bm25_retriever = BM25OpenSearchRetriever(
    client=aws_client,
    index_name=index_name,
    is_filter=True,
    filter01=filter01,
    filter02=filter02,
    k=5 # return up to k
)

In [47]:
chain = load_qa_chain(
    llm=llm_text,
    chain_type="stuff",
    prompt=PROMPT,
    verbose=True
)

In [50]:
search_keyword_result = opensearch_bm25_retriever.get_relevant_documents(query)

answer = chain.run(
    input_documents=search_keyword_result,
    question=query
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Here is the context, inside <context></context> XML tags.    

<context>
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  
- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록
 
 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.

ask: 인터넷뱅킹 비밀번호를 잊어버렸습니다.
Information: 인터넷뱅킹 로그인시 사용되는 인증서 암호를 잊어버리셨다면, 영업점 방문없이 인증서 재발급을 통해 인증서 암호를 다시 설정하실수 있습니다. 당행에서 발급한 인증서인 경우 당행 홈페이지에서 재발급하시면 됩니다. 신한은행 홈페이지 인증센터로 접속하시면 금융인증서와 공동인증서구 공인인증서로 메뉴가 분리되어 있으므로 해당 메뉴를 선택해서 접속해 주시면 됩니다. 타기관에서 발급된 인증서인 경우엔 해당기관에서 재발급후, 당행 홈페이지 타기관 공인인증서 등록/해제 메뉴에서 등록 후 이용하시면 됩니다.

ask: 인터넷뱅킹에서 모바일OTP 인증요청 했는데 푸시메세지가 오지 않아요. 어떻게 해야하나요?
Information: 신한 쏠(SOL) 푸시알림 수신에 동의를 하신경우 푸시를 받을수 있습니다. 경로 : 신한 쏠(SOL) 로그인 > 전체메뉴 > 설정/인증  > 일반 > 알림 > 금융정보 알림 항목을 수신동의로 설정 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍

### 키워드 검색 결과 (search_keyword_result)
bm25 score는 max_value로 normalization 되어 있음 (score range 0 - 1)

In [51]:
show_context_used(search_keyword_result)

-----------------------------------------------
1. Chunk: 279 Characters
-----------------------------------------------
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다.
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록
- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록

 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
metadata:
 {'source': '기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.', 'score': 1.0}
-----------------------------------------------
2. Chunk: 313 Characters
-----------------------------------------------
ask: 인터넷뱅킹 비밀번호를 잊어버렸습니다.
Information: 인터넷뱅킹 로그인시 사용되는 인증서 암호를 잊어버리셨다면, 영업점 방문없이 인증서 재발급을 통해 인증서 암호를 다시 설정하실수 있습니다. 당행에서 발급한
인증서인 경우 당행 홈페이지에서 재발급하시면 됩니다. 신한은행 홈페이지 인증센터로 접속하시면 금융인증서와 공동인증서구 공인인증서로 메뉴가 분리되어 있으므로 해당 메뉴를 선택해서
접속해 주시면 됩니다. 타기관에서 발급된 인증서인 경우엔 해당기관에서 재발급후, 당행 홈페이지 타기관 공인인증서 등록/해제 메뉴에서 등록 후 이용하시면 됩니다.
metadata:
 {'source': 'Information: 인터넷뱅킹 로그인시 사용되는 인증서 암호를 잊어버리셨다면, 영업점 방문없이 인증서 재발급을 통

## (3) OpenSearch Hybrid 검색

OpenSearch Hybrid 는 아래와 같은 방식으로 작동합니다.
- (1) "Vector 서치" 하여 스코어를 얻은 후에 표준화를 하여 스코어를 구함. 
    - 전체 결과에서 가장 높은 스코어는 표준화 과정을 통하여 스코어가 1.0 이 됨.
- (2) Keyword 서치도 동일하게 함.
- (3) Reciprocal Rank Fusion (RRF) 기반 Re-rank
    - Paper: https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
    - Desc: https://medium.com/@sowmiyajaganathan/hybrid-search-with-re-ranking-ff120c8a426d
    - **RRF의 경우 score가 아닌 ranking 정보를 활용, 때문에 score normalization이 필요 없음**
    - ![rrf.png](./img/rrf.png)

RRF는 langchain에서 "Ensemble Retriever" 이름으로 api를 제공합니다. 
- https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble


### Ensemble retriever 정의
- https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble
- Rank constant (param "c")
    - This value determines how much influence documents in individual result sets per query have over the final ranked result set. A higher value indicates that lower ranked documents have more influence. This value must be greater than or equal to 1. Defaults to 60.
    - 숫자 높을 수록 낮은 랭크의 문서가 더 중요시 된다

In [52]:
from langchain.retrievers import EnsembleRetriever

In [53]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[opensearch_bm25_retriever, opensearch_semantic_retriever],
    weights=[0.8, 0.2],
    c=100,
    k=5
)

In [54]:
chain = load_qa_chain(
    llm=llm_text,
    chain_type="stuff",
    prompt=PROMPT,
    verbose=True
)

In [55]:
%%time

search_hybrid_result = ensemble_retriever.get_relevant_documents(query)

answer = chain.run(
    input_documents=search_hybrid_result,
    question=query
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Here is the context, inside <context></context> XML tags.    

<context>
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  
- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록
 
 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.

ask: 인터넷뱅킹에서 모바일OTP 인증요청 했는데 푸시메세지가 오지 않아요. 어떻게 해야하나요?
Information: 신한 쏠(SOL) 푸시알림 수신에 동의를 하신경우 푸시를 받을수 있습니다. 경로 : 신한 쏠(SOL) 로그인 > 전체메뉴 > 설정/인증  > 일반 > 알림 > 금융정보 알림 항목을 수신동의로 설정 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.

ask: 인터넷뱅킹 비밀번호를 잊어버렸습니다.
Information: 인터넷뱅킹 로그인시 사용되는 인증서 암호를 잊어버리셨다면, 영업점 방문없이 인증서 재발급을 통해 인증서 암호를 다시 설정하실수 있습니다. 당행에서 발급한 인증서인 경우 당행 홈페이지에서 재발급하시면 됩니다. 신한은행 홈페이지 인증센터로 접속하시면 금융인증서와 공동인증서구 공인인증서로 메뉴가 분리되어 있으므로 해당 메뉴를 선택해서 접속해 주시면 됩니다. 타기관에서 발급된 인증서인 경우엔 해당기관에서 재발급후, 당행 홈페이지 타기관 공인인증서 등록/해제 메뉴에서 등록 후 이용하시면 됩

In [56]:
show_context_used(search_hybrid_result)

-----------------------------------------------
1. Chunk: 279 Characters
-----------------------------------------------
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다.
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록
- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록

 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
metadata:
 {'source': '신한은행', 'row': 3, 'type': '인터넷뱅킹', 'timestamp': 1696832766.2917683}
-----------------------------------------------
2. Chunk: 225 Characters
-----------------------------------------------
ask: 인터넷뱅킹에서 모바일OTP 인증요청 했는데 푸시메세지가 오지 않아요. 어떻게 해야하나요?
Information: 신한 쏠(SOL) 푸시알림 수신에 동의를 하신경우 푸시를 받을수 있습니다. 경로 : 신한 쏠(SOL) 로그인 > 전체메뉴 > 설정/인증  > 일반 > 알림
> 금융정보 알림 항목을 수신동의로 설정 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
metadata:
 {'source': '신한은행', 'row': 11, 'type': '인터넷뱅킹', 'timestamp': 1696832766.2917902}
-----------------------------------------------
3. Chunk: 313 Characters
---------